In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.interpolate import griddata
from PIL import Image
from tqdm import tqdm




# Define the grid size for interpolation grid
grid_size = 15000
# grid_size = 1000

# paths to the .npy file
npy_folders_path = [r'.\..\OneDrive_2025-03-03\LED_data_for prediction\Dataset\Data_preprocessing_2D_CNN\Data_All\Data\SAC105',
	r'.\..\OneDrive_2025-03-03\LED_data_for prediction\Dataset\Data_preprocessing_2D_CNN\Data_All\Data\SAC305',
	r'.\..\OneDrive_2025-03-03\LED_data_for prediction\Dataset\Data_preprocessing_2D_CNN\Data_All\Data\Senju',
]

# path to the directory to save the images
image_folder_path = r'.\images'








for npy_folder_path in npy_folders_path:
	# Create the directory if it doesn't exist to save the images
	os.makedirs(os.path.join(image_folder_path, os.path.basename(npy_folder_path)), exist_ok=True)

	# Get a list of all .npy files in the folder
	npy_files = [os.path.join(npy_folder_path, file) for file in os.listdir(npy_folder_path) if file.endswith('.npy')]


	for npy_file in tqdm(npy_files):
		# print(f"Processing {os.path.basename(npy_file)} file...")

		# define the path to save the greyscale image
		greyscale_image_save_path = os.path.join(image_folder_path, os.path.basename(npy_folder_path), f'grayscale_{os.path.basename(npy_file)[:-4]}_{grid_size}x{grid_size}.png')


		try:
			arr = np.load(npy_file)
			df = pd.DataFrame(arr, columns=['X Location (µm)', 'Y Location (µm)', 'Z Location (µm)', 'NLCREQ (µm/µm)'])
			
			# Extract each column into separate variables
			X_Location = df['X Location (µm)']
			Y_Location = df['Y Location (µm)']
			Z_Location = df['Z Location (µm)']
			NLCREQ = df['NLCREQ (µm/µm)']
			
		except Exception as e:
			print("Failed to read the file:", e)





		# # Plot XY plane
		# plt.figure(figsize=(8, 6))
		# plt.plot(X_Location, Y_Location, 'bo', markersize=3)
		# plt.xlabel('X Location (µm)')
		# plt.ylabel('Y Location (µm)')
		# # plt.title('Y vs X')
		# plt.gca().set_aspect('equal', adjustable='box')
		# plt.grid(True)
		# plt.show()





		# Group by X and Y Location and calculate the mean of NLREQ
		grouped_data = df.groupby(['X Location (µm)', 'Y Location (µm)'])['NLCREQ (µm/µm)'].mean().reset_index()

		# Round 'NLCREQ' values to 4 decimal places
		grouped_data['NLCREQ (µm/µm)'] = grouped_data['NLCREQ (µm/µm)'].round(4)

		# # Plotting
		# plt.figure(figsize=(8, 6))
		# plt.scatter(grouped_data['X Location (µm)'], grouped_data['Y Location (µm)'],
		# 			c=grouped_data['NLCREQ (µm/µm)'], cmap='viridis')
		# plt.colorbar(label='NLCREQ (µm/µm)')
		# plt.xlabel('X Location (µm)')
		# plt.ylabel('Y Location (µm)')
		# # plt.gca().invert_yaxis()

		# plt.title('Scatterplot - XY (NLCREQ averaged)')
		# plt.grid(True)
		# plt.show()





		# Plotting the Averaged data and interpolated data side by side
		# fig, axes = plt.subplots(1, 2, figsize=(20, 10))

		# # Get the min and max for setting the same scale
		# x_min, x_max = grouped_data['X Location (µm)'].min(), grouped_data['X Location (µm)'].max()
		# y_min, y_max = grouped_data['Y Location (µm)'].min(), grouped_data['Y Location (µm)'].max()
		# nlcreq_min, nlcreq_max = grouped_data['NLCREQ (µm/µm)'].min(), grouped_data['NLCREQ (µm/µm)'].max()



		# # Averaged data plot
		# axes[0].scatter(grouped_data['X Location (µm)'], grouped_data['Y Location (µm)'],
		# 							c=grouped_data['NLCREQ (µm/µm)'], cmap='viridis', vmin=nlcreq_min, vmax=nlcreq_max)
		# axes[0].set_title('Averaged NLCREQ Data')
		# axes[0].set_xticks([])
		# axes[0].set_yticks([])
		# axes[0].set_xlim(x_min, x_max)
		# axes[0].set_ylim(y_min, y_max)



		# Interpolated data plot
		# Define grid points based on the current file's data
		x_vals = np.linspace(grouped_data['X Location (µm)'].min(), grouped_data['X Location (µm)'].max(), grid_size)
		y_vals = np.linspace(grouped_data['Y Location (µm)'].min(), grouped_data['Y Location (µm)'].max(), grid_size)
		X, Y = np.meshgrid(x_vals, y_vals)

		# Interpolate NLCREQ values onto the grid
		NLCREQ_interp = griddata((grouped_data['X Location (µm)'], grouped_data['Y Location (µm)']),
									grouped_data['NLCREQ (µm/µm)'], (X, Y), method='linear')


		# # plot it
		# img = axes[1].imshow(NLCREQ_interp,
		# 						extent=(x_min, x_max, y_min, y_max),
		# 						origin='lower',
		# 						aspect='auto',
		# 						cmap='viridis', vmin=nlcreq_min, vmax=nlcreq_max)
		# axes[1].set_title('Interpolated NLCREQ Data')
		# axes[1].set_xticks([])
		# axes[1].set_yticks([])
		# # axes[1].invert_yaxis()  # Invert the y-axis

		# # Add a colorbar to the right side of the plots
		# fig.colorbar(img, ax=axes, orientation='vertical', fraction=0.02, pad=0.04, label='NLCREQ (µm/µm)')

		# # Display the plots
		# plt.show()





		# Normalise NLCREQ dimension and save it as grayscale image of the interpolated data

		# handle nans and infinites in the array
		NLCREQ_interp = np.nan_to_num(NLCREQ_interp)  # Replace NaNs and infinities with 0 and max int

		# convert third dimension as grey scale
		normalized_max = np.max(NLCREQ_interp)
		if normalized_max == 0:
			normalized_img = np.zeros_like(NLCREQ_interp, dtype=np.uint8)
		else:
			normalized_img = (NLCREQ_interp / normalized_max * 255).astype(np.uint8)

		# save image using Matplotlib library
		plt.imsave(greyscale_image_save_path, normalized_img, cmap='grey', format='png', dpi=600)
		# print(f"Image saved: {greyscale_image_save_path}")